# Get info, set storage & delete an instance

In [ ]:
import lamindb as ln

## Get info

In [ ]:
ln.setup.load("mydata")

In [ ]:
ln.setup.info()  # CLI: lamin info

## Set storage

In [ ]:
import laminci

pgurl = laminci.db.setup_local_test_postgres()

In [ ]:
ln.setup.init(storage="./storage_1", db=pgurl)

Local storage:

In [ ]:
ln.setup.set.storage("./storage_2")  # CLI: lamin set --storage ./storage_2

In [ ]:
from pathlib import Path

assert ln.setup.settings.storage.root_as_str == f"{Path.cwd()}/storage_2"

Cloud storage:

In [ ]:
ln.setup.set.storage("s3://lndb-setup-ci")  # lamin set --storage s3://lndb-setup-ci

See an overview:

In [ ]:
ln.select(ln.Storage).df()

In [ ]:
assert ln.setup.settings.storage.is_cloud
assert ln.setup.settings.storage.root_as_str == "s3://lndb-setup-ci"
# root.fs contains the underlying fsspec filesystem
assert (
    ln.setup.settings.storage.root.fs.cache_regions  # set by lamindb to True for s3 by default
)

You can set any additional `fsspec` filesystem arguments for cloud storage, such as `profile` or `cache_regions` (for s3 only), for example:

In [ ]:
ln.setup.set.storage("s3://lndb-setup-ci", cache_regions=False)

In [ ]:
# test cache_regions
assert not ln.setup.settings.storage.root.fs.cache_regions
# test setting storage not by owner
ln.setup.login(
    "testuser2@lamin.ai", password="goeoNJKE61ygbz1vhaCVynGERaRrlviPBVQsjkhz"
)
ln.setup.set.storage("./storage_3")
assert ln.setup.settings.storage.root_as_str == f"{Path.cwd()}/storage_3"
!docker stop pgtest && docker rm pgtest

### Currently not possible: setting storage for SQLite instance

In [ ]:
ln.setup.login("testuser1")
ln.setup.load("mydata")

If you try to set the storage for an sqlite instance, an error message is returned:

In [ ]:
assert ln.setup.set.storage("mydata_storage_2") == "set-storage-failed"

## Delete an instance

### With local default storage

In [ ]:
ln.setup.init(storage="mydata-delete")

In [ ]:
ln.setup.delete("mydata-delete")  # CLI: lamin delete mydata

In [ ]:
from lndb.dev._settings_store import instance_settings_file

settings_file = instance_settings_file("mydata-delete", "testuser1")
assert settings_file.exists() == False

### With remote default storage

In [ ]:
ln.setup.init(storage="s3://lndb-setup-delete-ci")

In [ ]:
ln.setup.delete("lndb-setup-delete-ci")